# Заметки о работе и проверке модуля планирования

1. [Как проверять работу планировщика на тестовых комплексах](#Как-проверять-работу-планировщика-на-тестовых-комплексах)
  1. [Какие комплексы есть](#Какие-комплексы-есть)
  2. [Где брать логи](#Где-брать-логи)
  3. [Доступ к веб-клиенту](#web)
  4. [На что смотреть для быстрой проверки](#На-что-смотреть-для-быстрой-проверки)
  5. [Разные нюансы при проверке результатов планирования](#Разные-нюансы-при-проверке-результатов-планирования)
2. [Из чего состоит расчет на серверах](#Из-чего-состоит-расчет-на-серверах)
3. [Как готовить планировщик для релизной сборки](#Как-готовить-планировщик-для-релизной-сборки)
  1. [Регламент релизов](#Регламент-релизов)
  2. [Как проверять планировщик для релиза](#Как-проверять-планировщик-для-релиза)
  3. [Батники и скрипты на сервере](#Батники-и-скрипты-на-сервере)
  4. [Как и куда выкладывать](#Как-и-куда-выкладывать)
  5. [Патчи](#Патчи)
4. [Регламент запуска планировщика на комплексах](#Регламент-запуска-планировщика-на-комплексах)
  1. [Расписание запусков](#Расписание-запусков)
  2. [Взаимодействие с УТХ](#Взаимодействие-с-УТХ)
  3. [Проблемы взаимодействия с УТХ](#Проблемы-взаимодействия-с-УТХ)
5. [ТрансСистемоТехника](#ТрансСистемоТехника)
6. [Показатели планирования](#Показатели-планирования)
  1. [Самые важные показатели](#Самые-важные-показатели)
  2. [Чуть менее важные показатели](#Чуть-менее-важные-показатели)
  3. [Разные эвристики и подгоны для получения хороших цифр](#Разные-эвристики-и-подгоны-для-получения-хороших-цифр)
  4. [Что может влиять на показатели](#Что-может-влиять-на-показатели)
7. [Люди в команде](#Люди-в-команде)
8. [Разное](#Разное)

## Как проверять работу планировщика на тестовых комплексах

### Какие комплексы есть

Всего есть четыре сервера (точнее, комплекса: каждый из них представляет из себя несколько серверов - но далее слова "сервер" и "комплекс" будут синонимичны, если только не будет уточнений к "серверу"), на которых запускается наш планировщик:

1. **Боевой комплекс**. Это продакшен-сервер, находится в Иркутске (или в Екатеринбурге - тут я путаюсь, но это и неважно). На него ставятся релизные сборки, которые (в идеале) выпускаются раз в две недели. Между выпуском версии и установкой сборки проходит какое-то время, пока ее проверяют сотрудники в Иркутске, так что вполне могут быть случаи, что на бою оказывается какая-то очень древняя версия планировщика. За боем следят тестировщики, так что нам каждый день туда заходить не обязательно. Если на бою выявляется какой-то критический баг, требующий новой версии планировщика, то его надо исправлять патчем (= отправкой новой версии админам). Про патчи - см. ниже раздел ["Патчи"](#Патчи).

2. **ТК1** ("первый тестовый/тренажерный комплекс"). У него статус очень похож на статус боевого: туда тоже ставятся релизные сборки, где они проверяются сотрудниками в Иркутске перед выкладкой на бой. Механизм нашей работы с ним примерно аналогичен бою:  тоже обновления только патчами, очень тщательно следить не стоит (если только по требованию руководства).

3. **ТК2** ("второй тестовый/тренажерный комплекс"). Это наш тестовый комплекс, где мы проверяем сборки перед релизом. Именно на него, в основном, смотрят технологи и проверяют цифры, подвязку, показатели и пр. Его надо мониторить (я обычно это делаю по утрам, сразу после прихода) просто на предмет того, идут ли расчеты планировщика, все ли в порядке с данными (см. раздел ["На что смотреть для быстрой проверки"](#На-что-смотреть-для-быстрой-проверки)).

4. **Нижегородка**. Это отладочный комплекс, который расположен в офисе НИИАС на Нижегородке. В отличие от остальных комплексов, сюда могут свободно ходить разработчики, подкладывать свои файлы и скрипты, что-то подкручивать и тестировать (в рамках разумного). С выпуском новых официальных релизов версия на Нижегородке (иногда можно встретить сокращение НК = "нижегородский комплекс" или НН) обновляется. 

### Где брать логи

#### Лог обмена данными

Главное наше оружие при проверке каких-то ошибок в планировании - это лог обмена данными между планировщиком и Вектором, файл `jason-FullPlannerPlugin.log`. С ТК1, ТК2 и НК его можно достать самостоятельно. Для этого нужен компьютер из сети СТТ - и такой компьютер у нас есть, это НИИАСовский компьютер (с одним монитором), который стоит под моим столом справа. На нем надо открыть программу `WinSCP` (значок есть на рабочем столе), в появившемся окне выбрать один из серверов (надо смотреть на названия, они подписаны: TK1, TK2, NIZH), залогиниться. Откроется обычное двухпанельное окно файлового менеджера, справа будет файловая система на соответствующем сервере планировщиков, там можно найти нужный файл и скопировать на этот локальный компьютер. Дальше можно на флешке переносить файл куда угодно.

Логи лежат по пути `.../rvec/server/bin` (перед `rvec` может быть какой-то еще фрагмент пути). В этой папке всегда находится один незаархивированный файл `jason-FullPlannerPlugin.log` и несколько заархивированных `jason-FullPlannerPlugin*.zip`. Вместо звездочек в название архива подставляется **реальное время запуска самого расчета планировщика**. Подробнее про то, что это за время - см. раздел ["Из чего состоит расчет на серверах"](#Из-чего-состоит-расчет-на-серверах).

#### Логи отсевов.

Логи модуля отсевов (разработкой модуля и логированием занимался Сергей Варанкин) лежат на тех же серверах по пути `.../rvec/server/bin/log/planner_filters`. Там нас интересуют два файла:

* `otsev_detail.csv` - это последние операции со всеми поездами, локомотивами и бригадами, которые были выбраны исходным SQL-запросом (потом к этим бригадам были применены правила отсевов, некоторые бригады отсеялись). Про формат этого файла, думаю, подробнее напишет Сергей Варанкин.
* `otsev_uth_detail.csv`. Это тот же файл, но в нем оставлены только операции по бригадам, переданным нам из УТХ на текущие сутки.

#### Логи самого планировщика.

Логи, которые создает сам планировщик при расчете, лежат в `.../rvec/server/bin/log/uni_planner_log`. 

#### Логи авторана.

В эти файлы пишется (крупноблочно) процесс хода расчета (см. ["Из чего состоит расчет на серверах"](#Из-чего-состоит-расчет-на-серверах)) с указанием времени, сколько заняла та или иная часть. Собственно работа планировщика - это строка вида *"... ЕПР: сформирован план ... поездов, ... локомотивов, ... бригад за ..."*. Из этой строки можно узнать, сколько объектов вернул планировщик (slot_train, slot_loco, slot_team) и сколько времени занял сам расчет. Все строки выше этой относятся к подготовке данных. 

Важный нюанс. Если в процессе подготовки данных случился какой-то эксепшен, то планировщик просто не запустится. Но в авторане появится строчка "Запуск планировщиков выполнен за ...". Это известный баг логирования, про который я писал очень давно. На самом деле планировщик в этом случае не запускается. Выявить эти случаи можно по соответствующему файлу `jason-FullPlannerPlugin.log`- в нем будет только шапка без каких-либо данных.

#### Логи подготовки данных.

Это сугубо векторовские логи, которые нам смотреть необязательно. Я их смотрел только для того, чтобы побыстрее выявить эксепшены, которые возникли при подготовке данных (они не запишутся ни в авторан, ни в лог отсевов, их можно выявить только через векторовский лог). Лог можно найти там же по пути `.../rvec/conf_fast_start/.netbeans-config/var/log`. Здесь нас интересуют файлы с именами вида `*Server_DEV_*` (вместо * может стоять что угодно). Этих файлов может быть несколько (они создаются в начале суток и каждый раз при перезапуске сервера планировщиков). Надо (по дате изменения) выбрать нужный, затем по таймстемпам найти нужный временной фрагмент и начать его листать (лог большой, там много мусора), выискывая глазами всякие стектрейсы. Иногда можно что-то найти, но иногда ошибка спрятана глубоко и быстро визуально не обнаруживается.

<a id='web'></a>
### Доступ к веб-клиенту

На результаты планирования можно посмотреть через веб-клиент. Там можно по каждой станции посмотреть все поезда, которые мы запланировали к прибытию или отправлению на эту станцию, посмотреть время прибытия/отправления, локомотивы и бригады, с которыми поезд прибывал и отправлялся. Технологи только так и проверяют результаты - так что и нам придется это использовать, чтобы как-то проверить сказанное технологами. 

Для каждого из комплексов (см. выше) на СТТ-компьютере (есть подозрение, что веб-клиенты работают и из НИИАСовской сети - например, с Бориного компьютера тоже должен быть доступ) в хроме установлена закладка (просто см. по названию). Заходить можно под логином/паролем tsyganova/tsyganova. Далее в верхнем меню надо нажать кнопку "Планирование", выбрать нужную станцию, нужный план (выпадающий список "Версия") и нажать кнопку "Обновить". Назначение остальных фильтров интуитивно понятно (после выбора надо снова нажимать "Обновить"). 

Про возможности формы отображения планов лучше расскажет Оля Цыганова. Кратко скажу, что в таблице три секции: справа отображается информация по поезду на начало планирования (номер, индекс, вес, длина, последняя операция), далее - информация по планируемому прибытию поезда на выбранную станцию (время, локомотив, бригада), далее - информация по отправлению поезда с выбранной станции. 

Если вы не видите какого-то поезда на форме, то надо:
* проверить фильтр "Чет/нечет" - возможно, поезд едет в нечетную сторону, а выбран фильтр "ЧЕТ".
* проверить фильтр "Интервал". В нем выбирается горизонт (от начала планирования) для "обрезания" плана. Если выбран интервал "6 часов", то будут показаны операции прибытия и отправления, запланированные не позднее чем через 6 часов от времени начала планирования.
  * если операция с поездом планируется более чем через 24 часа от начала планирования, то увидеть его в форме невозможно.

### На что смотреть для быстрой проверки

Быстро оценить, что происходит, можно просто по размеру лога `jason-FullPlannerPlugin.log` или соответствующего архива. Возможны четыре очевидные ситуации:

1. Лог имеет размер 35...45 Мб (архив - 3.5...4 Мб). Тут все в порядке - данные в планировщик передались, расчет был завершен. С технической стороны тут все в порядке, далее имеет смысл изучать конкретные планы и логи.
2. Лог имеет размер 15...20 Мб (архив - 1...2 Мб). Это означает, что в планировщик были переданы не все данные. Скорее всего, была передана только нормативно-справочная информация, а вот актуальных данных по поездам, локомотивам и бригадам вообще нет. Такое чаще всего происходит, если отвалился поток данных из АСОУП в онтологию. Планировщик, скорее всего, в этом случае вернет сколько-то поездов (это будут фейковые поезда по заданиям `task` на поезда своего формирования), 0 локомотивов и 0 бригад. Что делать: администраторам чинить поток данных.
3. Лог имеет размер несколько килобайт (архив - несколько сот байт). Это означает, что данные в планировщик вообще не были переданы. При этом в логе будет только шапка - и больше ничего. Это свидетельствует о том, что случился необработанный эксепшен в процессе подготовки данных для планировщика, сам процесс не был завершен, данные не были сформированы. **Важное замечание**: при этом в логе авторана все равно появится строчка "Запуск планировщиков выполнен за ...". Это известный баг логирования, про который я писал очень давно. На самом деле планировщик в этом не запускается. Что делать: кому-то из группы онтологии искать и исправлять ошибку.
4. Лога (архива) нет вообще. Это означает, что сломался процесс запуска планировщика по расписанию. Такое иногда происходит из-за какой-то сложной ошибки вроде переполнения памяти и т.п. Что делать: кому-то из группы онтологистов или администраторов искать и исправлять ошибку, перезапускать комплекс.

### Разные нюансы при проверке результатов планирования
#### Индексы поезда

На форме в веб-клиенте указываются индексы поездов в формате 4-3-4. По ним эти поезда надо искать в логах. Соответствие id поезда и индекса пишется в начале лога (блок `"Поезда (obj_id = индекс поезда; номер поезда; источник):"`, обычно начинается со строчки 363 лога или около того). Проблема в том, что в логе пишутся индексы в 15-значном формате. Правило для поиска такое: если индекс поезда идет в формате `ABCD-EFG-HIJK` (4-3-4), то соответствующий ему 15-значный индекс будет таким: `ABCD0*EFGHIJK0*`, где вместо * может стоять какая-то любая цифра (после D и K идут нули).

Если кто-то из технологов говорит, что поезда с каким-то индексом нет в форме (хотя он должен быть), то надо поискать в логах поезд с точно таким же индексом, а также поезд, у которого совпадают только первые две части (4-3...), а "хвост" (последние 4 цифры - `HIJK` - другой). Такое вполне может быть, у поездов часто меняются концовки индексов (концовка индекса представляет собой 4 цифры ЕСР-кода станции назначения, станция назначения поезда может меняться по ходу следования).

Если для проверки использовать не файл с логом, а мои тестовые скрипты, то там все проще: индексы преобразуются при парсинге логов, в файле `train_info.csv` (который создается после завершения работы скрипта `read.py`) будет колонка `ind434`, где будет индекс в формате 4-3-4.

#### Фамилии бригад

На формах в веб-клиенте **не надо** искать бригады по фамилиям. Дело в том, что в этих фамилиях перемешаны кириллица и латиница, поэтому можно легко не найти бригаду, хотя она есть. Для поиска надо использовать табельный номер.

#### Сменно-суточное планирование

На форме планирования можно переключиться на горизонт "сменно-суточное". По сути, это тот же план, возвращенный планировщиком, но агрегированный, без конкретики по поездам/локомотивам/бригадам. В нем просто посчитано количество тех или иных поездов, локомотивов или бригад (подробнее см. форму) - отправленных или принятых, ушедших на ТО, ушедших или вышедших с отдыха и т.п. Количества посчитаны по каждому часу ж/д суток, посчитаны суммы по полусуткам и суткам. 

В основном, сменно-суточное планирование нужно для того, чтобы правильно оценить количество бригад, которое должно выйти на работу в каждом депо в следующие сутки. Эти количества передаются планировщику УТХ, планировщик УТХ подбирает конкретные бригады под это количество и возвращает пофамильный список. При следующем расчете этот список будет передан в наш планировщик (эти бригады будут переданы с признаком `uth(1)`) - и их надо будет планировать в первую очередь (этот механизм уже реализован).

Форма сменно-суточного планирования нуждается в тестировании - я не могу сейчас ручаться, что цифры на ней считаются абсолютно правильно и что они полностью соответствуют текущему плану.

#### Обновления иногда не переносятся


Это пункт "немного в сторону", но про него надо помнить. Из-за того, что у нас, по сути, четыре разных комплекса, то иногда бывает сложно уследить, какие доработки и обновления на каких комплексах установлены. Например, какой-то баг во входных данных был исправлен на НК, но не был добавлен в релизные метки (см. ниже) или не был добавлен патчем на ТК1-2 (тоже см. ниже). Потом случилось обновление комплексов - и эта доработка отовсюду исчезла. Надо это держать в голове и воспринимать спокойно. Если что, версию планировщика всегда можно посмотреть по метке в логах (поиск в логе по слову `version`) или по времени изменения jar-файла планировщика (он лежит по пути `server/bin/jason/`).

[В начало](#Заметки-о-работе-и-проверке-модуля-планирования)
## Из чего состоит расчет на серверах

Расчет планировщика на всех комплексах состоит из трех этапов.

#### Подготовка данных

Этот этап происходит на стороне онтологии, про него рассказывал Сергей Варанкин на своей лекции. Сначала происходит подготовка инфраструктуры (данных по станциям, участкам планирования и т.п.), затем - получение операций по поездам, локомотивам и бригадам и подготовка данных по ним. Если все работает нормально, то этот этап сейчас занимает примерно 25 минут. Если звезды на небе не сойдутся и на серверах начнутся какие-то проблемы (а они случаются с периодичностью), то время на подготовку данных сильно увеличивается - может достигать 3-4 часов. 

#### Расчет планировщика

Сам расчет начинается после того, как данные успешно (без ошибок) подготовлены и переданы в планировщик. Замечал, что время расчета на тестовых серверах немного отличается от времени расчета на нашем сервере (который в к.838) в большую сторону - ~17 минут вместо ~15. Над этим можно подумать (возможно, стоит попробовать как-то распараллелить расчет), но без фанатизма :)

#### Выгрузка результатов

После завершения расчета планировщика начинается этап выгрузки - точнее, на этом этапе происходит раскладка результатов планирования по таблицам в онтологии. Сейчас этот этап занимает примерно 7-10 минут.

Как можно посчитать, весь цикл расчета сейчас занимает от 45 минут. Перспективная идея "на подумать": как это время уменьшить.

[В начало](#Заметки-о-работе-и-проверке-модуля-планирования)
## Как готовить планировщик для релизной сборки

### Регламент релизов

Релизная сборка системы для последующей установки на боевой комплекс производится Славой Фроловым раз в две недели в ночь с четверга на пятницу. Последний релиз (на данный момент) будет собран 20.08. График может сдвигаться по разным причинам (например, кто-то накосячил с коммитами в ТФС и сборка собралась с ошибками), так что надо прислушиваться к разговорам на планерках на эту тему.

Поэтому к каждому релизному четвергу у нас должен быть стабильный работающий джарник. Не обязательно выкладывать новую версию к каждому релизу - если отлаженных доработок нет, то может сойти и предыдущая версия (тогда просто ничего в ТФС менять не надо).

### Как проверять планировщик для релиза

1. Собрать джарник.
2. Запустить его на имитационных данных ("от Войтенко"). Проверить, что проценты подвязки составляют 100-100-100-100 (эти проценты пишутся в консоли после завершения расчета). Запустить тестовые скрипты `_train_report`, `_loco_report`, `_team_report`. Проверить отчеты (см. раздел ["Тесты на имитационных данных"](#Тесты-на-имитационных-данных)).
3. Запустить джарник на реальных данных (лучше брать логи от какого-нибудь успешного расчета на запуске в 19:30, например, от каких-либо предыдущих суток с одного из тестовых комплексов). Проверить отчеты (см. раздел ["Тесты на реальных данных"](#Тесты-на-реальных-данных).
4. Провести "стресс"-тестирование (термин не совсем правильный, но пусть будет): запустить джарник последовательно на входных данных, на которых когда-то наблюдалось падение планировщика (необработанный эксепшен, аварийное завершение). Все такие логи собраны в папке `C:\work\test` на сервере.
5. Как запускать отчеты - см. описание проекта на гитхабе и раздел ["Батники и скрипты на сервере"](#Батники-и-скрипты-на-сервере).

#### Тесты на имитационных данных

Файл `jason-FullPlannerPlugin.log` с имитационными данными можно взять в ТФС по пути `Восточный полигон/Планировщики/ПланировщикиJason/docs/Имитационный полигон/jason-FullPlannerPlugin.rar`.

1. Поездной отчет:
  1. Проверить, что запланированы все поезда (слова "Всего ... реальных поездов ... не запланировано" в самом верху отчета).
  2. Проверить, что средние времена стоянок поездов на смену локомотивов не превышают 6 часов (раздел "Средние времена на смену локомотивов").
  3. Проверить, что средние времена стоянок поездов на смену бригад не превышают 4 часов (раздел "Средние времена на смену бригады").
  4. Проверить, что интервал между временами отправления поездов (строка в таблице "без учета резервных") не меньше 5 минут (раздел "Поиск поездов с интервалом между отправлениями меньше допустимого").
  5. Проверить, что для поездов с начальным состоянием `depart` первый участок запланированного маршрута совпадает с полученным во входных данных (раздел "Проверка соответствия первого участка в запланированном маршруте и исходного факта").
  6. Проверить, что в маршрутах поездов нет скачков по станциям и по времени назад (раздел "Проверка скачков по времени назад").
2. Локомотивный отчет:
  1. Посмотреть проценты подвязки (в самом начале отчета) - они могут отличаться от рассчитанных в анализаторе. Если это так, то проверить, что процент на горизонте 24 часа = 100%.
  2. Проверить количество пересылок локомотивов резервом, их не должно быть много (точный критерий мне сказать сложно, но на имитационных данных "много" - это больше 20). Раздел "Анализ локомотивов резервом".
  3. Проверить, что для локомотивов нет скачков по станциям и по времени назад (разделы "Проверка скачков по станциям" и "Проверка скачков по времени назад").
3. Бригадный отчет:
  1. Посмотреть проценты подвязки (в самом начале отчета) - они могут отличаться от рассчитанных в анализаторе. Если это так, то проверить, что процент на горизонте 24 часа = 100%.
  2. Проверить, что планировщик не запланировал фейковых бригад (раздел "Распределение реальных и фейковых бригад в результатах планирования").
  3. Проверить, что в маршрутах бригад нет скачков между станциями (раздел "Скачки по станциям на маршруте бригад").    

#### Тесты на реальных данных

1. Поездной отчет:    
  1. Может присутствовать некоторое (небольшое) количество незапланированных реальных поездов: они не планируются из-за ошибочных входных данных (до 2%).
  2. Среднее время стоянок на смену локомотивов не должно превышать 10-11 часов, на смену бригад: 4-5 часов (опционально, можно сделать после выкладывания джарника: проверить и разобрать очень длительные стоянки на смену локомотивов на станции Карымская (в отчете приводятся примеры)).
  3. "Поиск поездов с интервалом между отправлениями меньше допустимого" - допускаются небольшие ненулевые значения для строки "Без учета резервных" (до 20), но на смотреть на примеры в таблице ниже. Участки, на которых происходят такие коллизии, должны быть не на главном ходу (не содержать крупных станций - Иркутск, Красноярск, Карымская и т.п.). Например, если есть 10 коллизий на участках типа Высокогорная - Соллу или Находка - Хмыловский, то это терпимо. В противном случае надо проверять и исправлять.
  4. Остальные проверки - такие же, как для имитационных данных.
2. Локомотивный отчет:
  1. Проверить проценты подвязки. Значение на 6 часов должно быть не меньше 90, на 24 часа - не меньше 80 (в перспективе - исследовать случаи, когда проценты не 100, и исправить).
  2. В разделе "Проверка выезда локомотивов за пределы своих тяговых плеч" должны быть нули.
  3. Раздел "Проверка планирования ТО-2": должна присутствовать строка "Все ТО запланированы на правильных станциях", времена проведения ТО должны составлять 3-4 часа.
  4. Раздел "Проверка смены локомотивов на станциях обязательной смены": в строке "Всего ... поездов, у которых должна быть смена локомотивов на станциях обязательной смены, но не запланирована" должно быть значение 0 (в идеале) или какое-то очень небольшое (до 20). Если значение не 0, то джарник выкладыват можно, но надо исследовать случаи проезда мимо станции смены.
  5. Раздел "Анализ локомотивов резервом": количество отправок локомотивов резервом должно быть не очень большим (ориентировочно - 100-150); не должно быть большого количества локомотивов резервом с временем хода больше 15 часов ("большое" - это 20% и больше).
  6. "Проверка скачков по станциям", "Проверка скачков по времени назад" - так же, как и для имитационных данных.
3. Бригадный отчет:
  1. Проверить проценты подвязки. Они не должны быть меньше 90% с учетом фейковых бригад (на горизонте 24) и меньше 80% без учета фейковых.
  2. Проверить процент используемых фейковых бригад: он не должен быть больше 10%.
  3. Проверить использование реальных бригад по состояниям (столбцевая диаграмма в разделе "Расчет процента запланированных бригад от общего количества бригад на входе"). Основные состояния бригад на входе - 1, 3 и 4, по ним не должно быть существенного проседания в доле использованных бригад (существенное - это когда используется меньше половины переданных). 
  4. Раздел "Скачки по станциям на маршруте бригад": скачки могут быть из-за учета узловых станций, поэтому достаточно проверить, что таких скачков не очень много (меньше 100).
  5. Раздел "Проверка случаев смены бригад не на допустимых станциях": в идеале там должен быть 0, но допускается небольшое отклонение (меньше 20). Смены бригад не на допустимых станциях надо затем исследовать.
  6. Разделы "Проверка отправления бригады не ранее времени явки", "Проверка планирования бригад до времени начала планирования", "Проверка подвязки негрузовых бригад" должны быть пустыми.
  7. Раздел "Проверка использования бригад после засылки пассажирами": идеальное значение = 0, допускаются небольшие отклонения. Криминальным являются случаи, когда бригада едет пассажиром, отдыхает, а затем снова едет пассажиром (для таких бригад в примерах в столбце `all_states` будет стоять `[0, 4]`).
  8. Раздел "Проверка сохранения предыдущих времен явок" - не должно быть существенных отклонений (больше 30% ошибочных бригад).    
  
** А теперь забудьте все, что написано выше. Все тесты должны проходить без ошибок - это наша глобальная цель**


### Батники и скрипты на сервере

На сервере есть папка `C:\work`, из которой можно делать сборку джарника, запуск, построение отчетов.

1. Сборка джарника делается батником `build.bat`. Он очищает папку `C:\work\UniversalPlanner`, клонирует проект из гита в нее же, собирает его maven'ом и копирует готовый джарник в `C:\work`. В процессе работы потребуется для пользователя гита `jenkins` ввести пароль - `jenkins123`.
2. Запуск джарника делается батником `only_run.bat`. Он просто запускает джарник `C:\work\JADEPlanner.jar` с входными данными из файла `C:\work\jason-FullPlannerPlugin.log`.
  1. Батник `only_run_trace.bat` сделает все то же самое, но с уровнем логирования TRACE. Это может понадобиться для отладки.
  2. Чтобы сразу собрать и запустить, можно использовать батник `build_n_run.bat` - он просто делает пп.1 и 2 по порядку.  
3. Построение отчетов (после завершения работы джарника) делается батником `create_reports.bat`. Он сначала создает файл
`log_for_analysis.log`, в котором объединяет входные данные из `C:\work\jason-FullPlannerPlugin.log` и результаты планирования из `C:\work\log\uni_planner_log\%Folder%\RESULTS.log` (вместо %Folder% будет подставлена последняя папка по алфавиту = по дате создания). Затем на этом файле запускается скрипт `read.py`, который парсит лог и создает нужные csv-шки, а потом последовательно запускаются все тестовые скрипты.
4. "Стресс" тестирование можно запустить батниками `build_n_run_full_test.bat` или `only_run_test.bat` (первый еще и соберет проект из гита, второй - просто запустит джарник). Они последовательно запускают планировщик на всех логах из папки `C:\work\test`, после завершения (весь процесс займет около 2 часов) надо просмотреть консоль на предмет количества запланированных поездов/локомотивов/бригад. Если на каком-то расчете будет результат 0/0/0 - это ошибка (на каких-то расчетах может быть результат N/0/0, где N != 0 - это нормально).
5. Все тестовые скрипты (файлы с расширением *.py) должны лежать в папке `C:\work`.

### Как и куда выкладывать

#### TFS

После того как джарник проверен, его надо выложить в ТФС. Для этого надо создать структуру папок `server\bin\jason`, положить джарник по этому пути (то есть должен получиться путь `server\bin\jason\JADEPlanner.jar`) и заархивировать папку `server` в zip-архив. Архив переименовать в `YYYYMMDD_JADEPlanner.zip`, вместо `YYYYMMDD` должны быть год, месяц и дата выкладывания.

![Example_JADE_Planner](https://dl.dropboxusercontent.com/u/3333057/jadeplanner.png)

Далее надо этот архив выложить в ТФС по пути `Восточный полигон/Планировщики/ПланировщикиJason/JADEPlanner` (добавить элемент в папку, вернуть изменения, в комментарии перечислить реализованные исправления (в идеале - с номерами задач в JIRA)).

Коммит выполнится. После этого можно посмотреть его номер (правой кнопкой мыши на только что добавленном файле - "Посмотреть журнал" - столбец "Набор изменений"), он пригодится далее в разделе [Патчи](#Патчи).

#### Файл labels.txt

Чтобы все закомиченные изменения были собраны автосборкой Фролова и попали в релиз, надо этой автосборке явно сказать, какой архив с джарником использовать. Для этого в ТФС есть файл `Восточный полигон/Релизы и инструменты сборки/Автоматическая сборка/labels.txt`. Этот файл надо скачать, найти в нем строчку `rvec_file = Планировщики/ПланировщикиJason/JADEPlanner/20160818_JADEPlanner.zip` (здесь будет указан какой-то из предыдущих архивов планировщика) и изменить название файла на название только что закомиченного архива. Далее надо сохранить `labels.txt` и вернуть его в ТФС (комментарий к возврату - что-то вроде "Новая версия планировщика", этого достаточно).

Нюансы. Несмотря на то, что релиз собирается в ночь с четверга на пятницу, Фролов иногда требует, чтобы все файлы были добавлены в `labels` уже во вторник. Про это надо помнить (и по возможности делать), но если файл будет добавлен в четверг, ничего страшного не случится (надо только добавить его до 19:00).

### Патчи

Если потребуется обновить планировщик на тестовых комплексах между релизами, то для этого используется механизм патчей. Это сделано для того, чтобы разработчики не ставили хаотично свои обновления на все тестовые комплексы так, чтобы потом невозможно было найти концы, кто накосячил и чье изменение все обрушило. 

Патчи делаются так. На каждый релиз в джире заводится "задача для патчей" (ее пример можно посмотреть [здесь](http://jira.programpark.ru/browse/VP-6863)). Дальше разработчики коммитят свои изменения в ТФС (можно без добавления в labels) и создают в джире подзадачу к этой "задаче для патчей". В ней они описывают сделанные исправления, пишут, на какие комплексы надо это исправление поставить (чаще всего это ТК2, но бывают и все остальные), обязательно указывают номер коммита в ТФС и - опционально - задачу в джире по исправленному багу/новому функционалу. Задача назначается на кого-то из администраторов (лучше на руководителя группы администраторов Александра Рыбинцева, он переназначит на нужного человека). Когда администратор установил патч, он отписывается в этой задаче и переводит ее или на тестировщика (чаще всего это Бураева), или на автора (разработчика, выложившего патч). Пример подзадачи с патчем можно посмотреть [здесь](http://jira.programpark.ru/browse/VP-7220).

### Список изменений

После тестирования каждого релиза тестировщики и администраторы составляют список изменений, которые вошли в этот релиз (дальше этот список пойдет в Иркутск, чтобы там знали, что изменилось в планировщике). Чтобы правильно составить этот список, надо а) узнать у администраторов, для какой именно версии планировщика составляется этот список (чаще всего - это та версия, которая установлена на ТК1 или ТК2, можно посмотреть дату изменения `JADEPlanner.jar` на соответствующем сервере); б) посмотреть в ТФС историю коммитов, по комментариям к коммитам составить список изменений (поэтому важно писать комментарии).

[В начало](#Заметки-о-работе-и-проверке-модуля-планирования)
## Регламент запуска планировщика на комплексах

### Расписание запусков

На всех комплексах установлено расписание автоматических запусков планировщика. По этому регламенту планировщики запускаются каждые три часа, начиная с 16:30. Эти запуски происходят в режиме "Текущий" (параметр конфигурации во входных данных: `+config("bulk_planning",0)`). 16:30 - это именно время начала всего цикла расчета (см. раздел ["Из чего состоит расчет на серверах"](#Из-чего-состоит-расчет-на-серверах)), в этот момент начинается подготовка данных для планировщика. Собственно расчет планировщика начнется примерно через 30 минут (если все идет без ошибок) - и именно это время будет затем указано в названии архива с логом этого запуска. Если все проходит без ошибок, то весь цикл завершается примерно в 17:20 - 17:30, затем в 19:30 начинается новый расчет и т.д.

В 14:40 запускается еще один расчет планировщика - с типом "сменно-суточный" (параметр конфигурации во входных данных: `+config("bulk_planning",1)`), который рассчитывает объемные планы на следующие ж/д сутки. Так что предыдущий расчет (от 13:30) должен обязательно быть завершен к 14:40.

### Взаимодействие с УТХ

Это долготянущаяся и важная тема. Смысл вот в чем.

Расчет планировщика ОУЭР в 14:40 нужен преимущественно для того, чтобы узнать, сколько бригад понадобится для вывоза всех поездов на следующие сутки. После получения результатов планирования в онтологии производится подсчет количества запланированных на следующие сутки бригад (отдельно по каждому депо приписки), это рассчитанное количество передается в планировщик УТХ. Планировщик УТХ анализирует историю операций по всем бригадам данного депо приписки и выдает пофамильный список бригад, которые выйдут на работу в этом депо в следующие сутки (количество бригад в этом списке совпадает с количеством, рассчитанным после планирования ОУЭР). Далее эти бригады при следующих запусках планировщика ОУЭР будут передаваться в планировщик ОУЭР с признаком `uth(1)` - и эти бригады планировщик ОУЭР должен использовать при планировании в первую очередь.

Расписание запусков такое:
1. В 14:40 запускается сменно-суточный расчет ОУЭР. Он должен завершиться примерно к 15:30.
2. В 16:00 запускается расчет планировщика УТХ. Он должен завершиться к 16:30. **Это достаточно жесткое требование, поскольку в 16:30 запускается следущий расчет ОУЭР, который уже должен иметь во входных данных УТХ-бригады на следующие сутки**.
3. В 16:30 запускается текущий расчет ОУЭР, он уже использует новый список УТХ-бригад.
4. В 19:30 (и далее каждые три часа) снова запускается расчет ОУЭР. В него должны передаваться те же УТХ-бригады, при перерасчетах должны сохраняться времена явок бригад, запланированные на предыдущем расчете.

### Проблемы взаимодействия с УТХ

1. Сменно-суточный расчет ОУЭР (в 14:40) может не закончиться к моменту запуска УТХ (16:00) или вообще не состояться. Эти проблемы возникают на онтологической стороне - иногда может слишком затянуться (или вообще упасть с ошибкой) процесс подготовки данных для планировщика.
2. Планировщик УТХ может вообще не запуститься. Это происходит из-за того, что его надо как-то руками первый раз запустить на сервере после переустановки (или зависания? точно не знаю), а админы это не всегда делают. Отсутствие УТХ-расчета можно выявить по отсутствию файла `otsev_uth_detail.csv` в папке `server\bin\log\planner_filters`. В случае обнаружения об этом стоит проинформировать Хохлова и Бородулина (хотя они сами это могут отследить).
3. Планировщик УТХ может закончить работу после 16:30. Это приведет к тому, что в расчет планировщика ОУЭР в 16:30 не будут переданы новые УТХ-бригады. Это потенциально может быть большой проблемой - вот почему. Планировщик ОУЭР выполнит какой-то расчет в 16:30, запланирует явки каких-то не-УТХ-бригад в интервале с 21:00 до 0:00. Далее в 19:30 будет новый запуск, в котором (скорее всего) в планировщик ОУЭР таки будут переданы новые УТХ-бригады. И мы получим вот что: с одной стороны, ОУЭР должен приоритетно планировать эти новые УТХ-бригады, а с другой - он должен стремиться сохранить явки бригад, запланированные с 21:00 до 0:00, хотя это могут быть явки бригад, которых нет в списке УТХ. Получаем конфликт. Чтобы такого не было, надо обеспечить завершение расчета УТХ к 16:30.
4. Не все УТХ-бригады могут быть переданы в планировщик ОУЭР. Иногда такое случается, но такие случаи единичные (не передается 1-2 бригады). Чаще всего непередача происходит из-за того, что с УТХ-бригадой случилась какая-то нестандартная последняя операция, которой нет в списке "разрешенных" операций с бригадами в ОУЭР (если операции с бригадой нет в списке "разрешенных", то такая бригада просто отсеивается при подготовке данных для ОУЭР). Код последней операции (и другие причины отсевов УТХ-бригад) можно посмотреть в отчете `otsev_uth_detail.csv` в папке `server\bin\log\planner_filters`. Если такое случилось, то "плохую" операцию надо просто добавить в список разрешенных операций для УТХ-бригад (раньше это делал Сергей Варанкин).
5. Не все УТХ-бригады могут быть использованы планировщиком ОУЭР в планировании (правда, таких бригад немного - 4-5). Это может быть по нескольким причинам:
  1. Баги в планировщике. По-хорошему, надо бы исследовать каждый случай неиспользования на предмет багов.
  2. Изменилась поездная обстановка. Например, какой-то поезд планировался к отправлению, но при следующем расчете оказалось, что он уже уехал или еще не приехал на станцию. Тогда какая-то бригада может оказаться невостребованной.
  3. Не удалось сохранить предыдущую явку. Требования на сохранение времени предыдущей явки достаточно жесткие (на первые 3 часа). Поэтому если какой-то поезд немного задержится, то бригада уже не сможет быть на него подвязана (поскольку надо сохранить явку). Это связано с п.2.
  4. Во входных данных по бригаде были ошибки (например, передано некорректное местоположение бригады или не совпадают входные данные по бригаде и локомотиву, если бригада на начало планирования уже прикреплена к локомотиву). 

[В начало](#Заметки-о-работе-и-проверке-модуля-планирования)
## ТрансСистемоТехника

ТрансСистемоТехника (ТСТ) - это компания, которая делает еще один планировщик в ИСУЖТ. Этот планировщик будет рассчитывать прогнозное время формирования поездов на станциях. 

Сейчас в планировщик ОУЭР передается только **количество** поездов своего формирования - это поезда, которые будут сформированы на какой-либо станции в течение текущих и следующих железнодорожных суток. Эти поезда необходимо учитывать при планировании, поскольку под них требуются локомотивы и бригады. Но сейчас мы не знаем, когда именно (в какое время) эти поезда будут готовы к отправлению - поэтому в планировщик передается только количество и временной интервал планирования (обычно это половина суток), а дальше планировщик равномерно распределяет отправление этого количества в этом интервале. Эти количества поездов своего формирования передаются в сообщениях `task` (являясь как бы "заданием" на планирование).

Планировщик ТСТ обещает передавать для таких поездов своего формирования рассчитанное время готовности поезда к отправлению. На работе планировщика ОУЭР это никак не скажется: просто часть поездов своего формирования будет передаваться сообщениями `train_ready` с временем готовности поезда к отправлению когда-то в будущем. Соответственно, некоторые сообщения `task` (для станций, на которых есть данные от ТСТ) передаваться не будут. Но полностью отказываться от тасков нельзя, поскольку планировщик ТСТ будет работать (с нужной долей правильности) не на всех станциях.

[В начало](#Заметки-о-работе-и-проверке-модуля-планирования)
## Показатели планирования

Показатели планирования - это некоторые критерии, по которым мы можем сказать, насколько хороший план рассчитал наш планировщик. Эти показатели используются железнодорожниками для оценки работы (по факту - рассчитываются показатели для уже совершившихся поездок поездов/локомотивов/бригад), мы же рассчитываем эти показатели для нашего плана на следующие сутки. Показатели считаются в модуле "Анализатор" (его писал в свое время Андрей), требования на расчет этих показателей приведены в документе `Модуль анализа результатов планирования.docx`, который лежит в ТФС по пути `/Восточный полигон/Планировщики/ПланировщикиJason/docs`.

### Самые важные показатели

Самые важные показатели - это те, на которые больше всего смотрят железнодорожники.

#### Производительность локомотивов

Это вообще главное. Мои исследования про производительность можно посмотреть в блокноте `_metrics_report.ipynb` (на гитхабе). Значение производительности должно быть около 2300, у нас сейчас меньше. На производительность, в основном, влияют вес поезда (если средний вес уменьшается, то и производительность уменьшится) и стоянки поездов на смену локомотивов (чем они меньше, тем больше локомотивы делают полезной работы - повышается производительность).

#### Маршрутная и участковая скорость

Это средняя скорость движения поездов в сутки (формулы см. в документе). На нее тоже влияют длительные стоянки поездов на смену локомотивов и бригад (очевидно, что поезд должен больше ехать, чем стоять). Надо проверить, что в анализаторе при подсчете этого показателя НЕ участвуют локомотивы резервом (а участвуют только грузовые поезда).

#### Средний вес поезда

На него мы не очень можем влиять, данные о весе поездов приходят нам из внешних систем. Средний вес должен быть около 3500. Надо проверить, что:
* при подсчете среднего веса мы не берем поезда своего формирования (поскольку для них вес неизвестен);
* при подсчете производительности локомотивов мы для поездов своего формирования берем вес поезда = среднему весу реальных поездов.

### Чуть менее важные показатели

На них железнодорожники смотрят меньше, но сейчас эти показатели важны для оценки качества нашего плана

#### Среднее время стоянки для смены локомотива (бригады)

Одна из основных проблем планировщика сейчас - слишком большое время на стоянку поезда для смены локомотива и бригады. Это время подсчитывается в анализаторе и выводится в тестовом отчете по поездам. Его надо уменьшать. Целевые показатели (по словам технологов): 2-3 часа для смены локомотивов, 1-2 часа для смены бригад.

#### Проценты подвязки

Это проценты поездов, обеспеченных локомотивами, и локомотивов, обеспеченных бригадами. Эти показатели надо стремить к 100%. В идеале надо выбрать время и исследовать причины не 100%-ных подвязок. Для бригад надо также смотреть на процент фейковых бригад: он должен стремиться к 0. 

#### Полезный пробег и процент полезного использования локомотивов

Эти показатели связаны с производительностью: чем больше процент полезного использования локомотивов, тем больше производительность. 

### Разные эвристики и подгоны для получения хороших цифр

Производительность: отбрасывание коротких маршрутов, локомотивов резервом, локомотивов, которые вторую половину суток стоят без работы. Аналогичные подгоны для полезного пробега и процента полезного использования. Сейчас примерно это реализовано в анализаторе. В блокноте `_metrics_report.ipynb` есть небольшой анализ, как эти подгоны влияют на производительность. 

### Что может влиять на показатели

1. Длительные стоянки поездов на смену локомотивов и бригад.
2. Количество запланированных поездов по тасками (чем их больше - тем больше поездов в принципе - тем больше производительность локомотивов).
3. Количество локомотивов резервом (если их много, то падает процент полезной работы).

### Разница между скриптами и анализатором

Есть расхождения во многих показателях (некоторые - совсем небольшие, некоторые - существенные). Надо выделить время и проверить подсчет процентов подвязки и производительности в анализаторе и в скриптах.

[В начало](#Заметки-о-работе-и-проверке-модуля-планирования)
## Люди в команде

Руководство вы знаете, так что перечислю остальных.

### Технологи

Технологи ОУЭР: Павел Евгеньевич Войтенко, Николай Иванович Капустин, Слава Фрольцов, Аким Семенчук, Дмитрий Мищенко. С ними можно обсуждать разные технологические вопросы - как делать планирование в тех или иных случаях. Так сложилось, что больше всего планирование проверял Войтенко (и с ним я больше всего общался по возникающим вопросам). В случае проблем или вопросов можно идти к нему, а он или ответит сам, или перенаправит к кому-нибудь другому.

Технологи УТХ: Владимир Петрович Алтунин, Владислав Жемионис, Алексей (?) Симонов. Если они приходят и говорят, что надо срочно исправить что-то для взаимодействия ОУЭР и УТХ, не надо сразу бросаться делать то, что они говорят. В обязательном порядке согласовывать план работ с Немцовым и Цыгановой, а сами постановки - с технологами ОУЭР (Войтенко и Капустин в первую очередь).

### Аналитики

Ольга Цыганова, Александр Есаков, Руслан Ушаков. Оля больше работает над UI и разными технологическими процессами при планировании (во сколько запускать планировщики, какие диспетчеры согласовывают план, как и что отображать на формах, когда и что корректировать). Также Оля ведет общий план работ и список ошибок, замеченных технологами и перечисленных в протоколах поездок в Иркутск. Саша пишет постановки на разные доработки в самом планировании (например, он писал про учет в планировщике нормативных времен работы бригад и про использование данных от ТСТ). Руслан занимается общим контролем взаимодействия с УТХ (тоже ведет протоколы и планы, назначает встречи/показы и пр.).

### Программисты

Михаил Костенко, Сергей Варанкин (он скоро увольняется), Даниил Тугушев, Дмитрий Ежов, Анастасия Типцова. Миша - руководитель группы онтологистов. В общем случае все доработки, которые для нас должны реализовать на стороне онтологии (например, какие-то новые входные данные или какие-то исправления уже существующих) надо делать через Мишу - он передаст задачу нужному человеку. Сергей Варанкин занимался собственно подготовкой данных для планировщика, вместо него будет Даниил Тугушев. Дима Ежов ответственен за получение данных из внешних систем в онтологию. Настя Типцова делает подготовку данных для вывода в формы с результатами - то есть, на ней парсинг и преобразование результатов планирования (например, это значит, что расчет цифр для формы сменно-суточного планирования и сохранение времен явок бригад для их передачи в планировщик на следующем расчете - тоже к ней).

### Тестировщики

Сэржана Бураева, Эльмар Гасанов. Эльмар - новый сотрудник, руководитель отдела тестирования. Собственно тестированием планировщика занималась Сэржана (но это тестирование только через просмотр форм текущего и сменно-суточного планирования). Также Сэржана собирает список доработок, которые должны войти в новый релиз.

### Администраторы

Александр Рыбинцев, Сергей Аникин, Влад Патрушев, Сергей Ребровский, Валерий Дорофеев. Александр и Валерий - руководители администраторов. Валерий больше ответственен за "железную" часть (настроить логины/пароли для доступа куда-то, посмотреть состояние серверов), а Александр - за "софтовую" (например, его надо извещать при установке патчей). Мелкие вопросы можно решать напрямую с Аникиным (если надо достать логи с боевого сервера, например).

Задачи с патчами можно создавать на Рыбинцева с добавлением в наблюдатели Аникина.

[В начало](#Заметки-о-работе-и-проверке-модуля-планирования)
## Разное

Тут я собрал всякие небольшие вещи, которые не попали в другие разделы

### Что может быть причиной плохого планирования

Весовые нормы. Есть участки, на которых к ведущему локомотиву обязательно нужен еще и локомотив для толкания. Для такого участка у локомотива определенных серий будет в справочнике сильно занижен максимальный вес, который этот локомотив может везти. Например, на участке Турма - Вихоревка для локомотива серии 3ЭС5К в справочнике установлен максимальный вес 4200. Именно это значение передается в планировщик и приводит к тому, что более тяжелые поезда просто не могут быть вывезены ни одним локомотивом.

Как это обходить (не реализовывая планирование подталкивающих локомотивов) - я пока не знаю. Возможно, сработает такое. Сейчас, когда мы анализируем весь маршрут поезда и для определенного локомотива узнаем, удовлетворяет ли этот локомотив весовым нормам, мы проходим по всему маршруту поезда, для каждого участка планирования по сочетанию участок + серия локомотива + число секций определяем максимальный допустимый вес поезда на этом участке, затем из всего списка максимальных весов выбираем минимальный и сравниваем его с весом поезда: если вес поезда больше, чем этот "минимакс", то локомотив не сможет везти этот поезд. Так вот предложение: брать не минимальное значение из списка максимально допустимых весов, а медианное. Скорее всего, такая фигня с подталкиванием (как описано выше) будет лишь на нескольких участках из маршрута. Например, список максимально допустимых весов на маршруте может быть таким: [5800, 5800, 6000, 4200, 5800, 5800]. Тогда если мы будем брать медиану, то сможем обойти эти "подталкивающие выбросы". Разумеется, эту идею надо анализировать и исследовать.

### Участки обкатки

Иногда некоторые станции пропадают из участков обкатки. Есть мелкие станции и разные небольшие ответвления от главного хода, где это не так важно. Но если станция пропадает из какого-то крупного участка с большим количеством поездов, то это будет плохо - реальные бригады просто не смогут быть подвязаны на поезда, которые едут через этот участок. Для проверки можно использовать отчет `_input_report`, в нем в разделе "Покрытие участками обкатки бригад" выводится список участков планирования, которые не принадлежат ни к одному участку обкатки бригад. Если в этом разделе будет замечена какая-то крупная станция (Иркутск, Красноярск и пр.), то это повод обратиться к онтологистам за фиксом.

### Локомотивам прописывается несколько тяговых плеч

Сейчас тяговые плечи назначаются локомотивам динамически в онтологии. Набор тяговых плеч зависит от депо приписки локомотива, серии и его местоположения. Есть случаи, когда для локомотива во входных данных передается несколько тяговых плеч (чаще всего это бывает, если локомотив на начало планирования находится на граничной станции тяговых плеч - Карымская, Тайшет,...). Это может привести к тому, что мы запланируем локомотив не в ту сторону, в которую надо. Тут тоже нужно исследование с привлечением технологов.

### Файл со статистикой связанности

По просьбе Немцова я периодически контролирую связанность данных на входе планировщика. Связанность - это, например, проценты поездов, у которых во входных данных должен быть связанный с ними локомотив, но этого локомотива нет (это поезда, которые на начало планирования находятся на перегоне). И так для пар поезд-локомотив и локомотива-бригада в обе стороны. Параметры, которые надо посчитать, перечислены в документе `/Восточный полигон/Планировщики/ПланировщикиJason/docs/Статистика по связанности входных данных.xlsx`. 

Эти параметры считаются в отчете `_input_report`, раздел "Статистика по связанности для выгрузки в Excel". Цифры из строки "Input" - это как раз значения параметров (см. картинку ниже), в строке "Output" пишутся количества поездов, локомотивов и бригад на выходе планировщика (это надо для столбца "Выход" в экселевской табличке со статистикой). В строки "% подвязки локо" и "% подвязки бригад" в экселевской табличке надо дописать вручную значения процента подвязки локомотивов к поездам на горизонте в 6 часов и процент подвязки бригад к локомотивам на горизонте в 6 часов (значения можно посмотреть в анализаторе).

![Excel](https://dl.dropboxusercontent.com/u/3333057/excel.png)